## Preprocessing

In [66]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [67]:
# Check for inbalanced variable
for column in application_df.columns:
    print(f"Column: {column}")
    print(application_df[column].value_counts())
    print("\n")


Column: EIN
EIN
10520599     1
626274659    1
630475330    1
630416100    1
630357662    1
            ..
383880377    1
383876652    1
383874900    1
383871370    1
996086871    1
Name: count, Length: 34299, dtype: int64


Column: NAME
NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIG

#### **Evaluation**

Looking at the data above, we can immediately see that there are three columns that may be best to be selected to be dropped due to portraying largely inbalaced data (single value encompassing 95%+ of the data). These three columns; 'STATUS', 'SPECIAL_CONSIDERATIONS' and 'ASK_AMT' each in almost most certainty added only noise and no valuable contribution to the training data thus potentially contributing to the models lower accuracy scores.

In [68]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'], axis=1)

### Application Type

In [69]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_list = application_df.value_counts('APPLICATION_TYPE')

application_type_list

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [70]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_list[application_type_list < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

### Classification

In [71]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_list = application_df['CLASSIFICATION'].value_counts()

classification_list

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [72]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_list[classification_list < 100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

### **Evaluation**

Considering that the classification column started with around 71 unique values, using a threshold of 1000 narrowed it down to just 5 main categories. While that helped simplify things, everything else got grouped into an “other” category—which actually ended up ranking 4th overall, which make me think there’s probably still something meaningful hiding in those less frequent classes for the Neural Network to uncover.

Because of that, I reduced the threshold to 100 to allow for more specific classifications to be included in training. This should give the model a better chance to pick up on more nuanced patterns across a wider range of categories.

In [73]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

application_df = pd.get_dummies(application_df, columns=categorical_columns, dtype=int)
application_df

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [74]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [75]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [76]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim = len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,209 (55.50 KB)

 Trainable params: 14,209 (55.50 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [78]:
# Train the model
nn_model_fit = nn.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6903 - loss: 0.5972 - precision_6: 0.6894 - recall_6: 0.7536 - val_accuracy: 0.7224 - val_loss: 0.5629 - val_precision_6: 0.7117 - val_recall_6: 0.8023
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7352 - loss: 0.5512 - precision_6: 0.7293 - recall_6: 0.7982 - val_accuracy: 0.7265 - val_loss: 0.5612 - val_precision_6: 0.7251 - val_recall_6: 0.7811
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.7300 - loss: 0.5504 - precision_6: 0.7301 - recall_6: 0.7866 - val_accuracy: 0.7238 - val_loss: 0.5616 - val_precision_6: 0.7223 - val_recall_6: 0.7796
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.7380 - loss: 0.5430 - precision_6: 0.7384 - recall_6: 0.7911 - val_accuracy: 0.7242 - val_loss: 0.5645 - val_precision_6: 0.7195 - val_recall_6: 0.7877
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.7346 - loss: 0.5433 - precision_6: 0.7312 - recall_6: 0.

In [79]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 488us/step - accuracy: 0.7285 - loss: 0.5528 - precision_6: 0.7269 - recall_6: 0.7861
Loss: 55.28%, Accuracy: 72.85%, Precision: 72.69%, Recall: 78.61%


In [80]:
# Export our model to HDF5 file
nn.save('exported_models/AlphabetSoupCharity_Optimized.h5')

## Re-Compile, Re-Train and Re-Evaluate the Model with extra Layer

In [81]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim = len(X.columns)))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,257 (63.50 KB)

 Trainable params: 16,257 (63.50 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Compile the model
nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [83]:
# Train the model
nn2_model_fit = nn2.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7026 - loss: 0.5936 - precision_7: 0.6934 - recall_7: 0.7917 - val_accuracy: 0.7193 - val_loss: 0.5624 - val_precision_7: 0.7142 - val_recall_7: 0.7859
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7325 - loss: 0.5514 - precision_7: 0.7299 - recall_7: 0.7948 - val_accuracy: 0.7236 - val_loss: 0.5625 - val_precision_7: 0.7198 - val_recall_7: 0.7851
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7337 - loss: 0.5436 - precision_7: 0.7337 - recall_7: 0.7885 - val_accuracy: 0.7217 - val_loss: 0.5635 - val_precision_7: 0.7178 - val_recall_7: 0.7840
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7413 - loss: 0.5382 - precision_7: 0.7349 - recall_7: 0.8089 - val_accuracy: 0.7215 - val_loss: 0.5602 - val_precision_7: 0.7194 - val_recall_7: 0.7796
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.7387 - loss: 0.5423 - precision_7: 0.7315 - recall_7: 0.7997

In [84]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn2.evaluate(X_test_scaled, y_test, verbose=2)
print("Extra Hidden Dense Layer:")
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 460us/step - accuracy: 0.7279 - loss: 0.5512 - precision_7: 0.7239 - recall_7: 0.7916
Extra Hidden Dense Layer:
Loss: 55.12%, Accuracy: 72.79%, Precision: 72.39%, Recall: 79.16%


In [85]:
# Export our model to HDF5 file
nn2.save('exported_models/AlphabetSoupCharity_extraLayer_Optimized.h5')

## Re-Compile, Re-Train and Re-Evaluate the Model using tanh instead of relu

In [86]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim = len(X.columns)))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=64, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,209 (55.50 KB)

 Trainable params: 14,209 (55.50 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Compile the model
nn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),])

In [88]:
# Train the model
nn3_model_fit = nn3.fit(X_train_scaled, y_train, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7018 - loss: 0.5936 - precision_8: 0.7138 - recall_8: 0.7355 - val_accuracy: 0.7213 - val_loss: 0.5736 - val_precision_8: 0.7206 - val_recall_8: 0.7760
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7335 - loss: 0.5562 - precision_8: 0.7260 - recall_8: 0.8072 - val_accuracy: 0.7190 - val_loss: 0.5704 - val_precision_8: 0.7028 - val_recall_8: 0.8155
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7293 - loss: 0.5533 - precision_8: 0.7261 - recall_8: 0.7895 - val_accuracy: 0.7199 - val_loss: 0.5663 - val_precision_8: 0.7029 - val_recall_8: 0.8184
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7343 - loss: 0.5498 - precision_8: 0.7212 - recall_8: 0.8034 - val_accuracy: 0.7224 - val_loss: 0.5612 - val_precision_8: 0.7191 - val_recall_8: 0.7833
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7427 - loss: 0.5393 - precision_8: 0.7389 - recall_8: 0.8022 -

In [89]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn3.evaluate(X_test_scaled, y_test, verbose=2)
print("Tanh instead of Relu:")
print(f"Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%, Precision: {round(model_precision * 100, 2)}%, Recall: {round(model_recall * 100, 2)}%")

268/268 - 0s - 466us/step - accuracy: 0.7283 - loss: 0.5506 - precision_8: 0.7237 - recall_8: 0.7933
Tanh instead of Relu:
Loss: 55.06%, Accuracy: 72.83%, Precision: 72.37%, Recall: 79.33%


In [90]:
# Export our model to HDF5 file
nn3.save('exported_models/AlphabetSoupCharity_tanh_Optimized.h5')

### **Evaluation**

Even after dropping three low-value columns and testing different architectures—including an extra dense layer and switching to tanh activation—the model’s performance remained consistent, with our best version achieving around 72.83% accuracy and 79.33% recall. This consistency suggests that while those columns weren’t the main source of noise, they were definitely contributing some, as the scores held steady despite removing three full columns of information.

At this point, the bigger limitation seems to lie within the dataset itself. There may be underlying patterns that are too subtle, inconsistent, or imbalanced for the model to learn effectively with the features provided. Meaningful improvement will likely require deeper data exploration, more targeted feature engineering, or better-quality inputs overall.